# ASP.NET Benchmark Analysis 

This notebook highlights the steps associated with analyzing data from the ASP.NET benchmarks obtained using crank.

In [ ]:
#r "nuget: Microsoft.Diagnostics.Tracing.TraceEvent, 3.1.13"
#r "nuget: YamlDotnet"
#r "nuget: XPlot.Plotly"
#r "nuget: XPlot.Plotly.Interactive"
#r "nuget: Microsoft.Data.Analysis, 0.19.1"
#r "nuget: Newtonsoft.Json"

using Etlx = Microsoft.Diagnostics.Tracing.Etlx;
using Microsoft.Data.Analysis;
using Microsoft.Diagnostics.Tracing.Analysis.GC;
using Microsoft.Diagnostics.Tracing.Analysis;
using Microsoft.Diagnostics.Tracing.Parsers.Clr;
using Microsoft.Diagnostics.Tracing;
using System.Diagnostics;
using XPlot.Plotly;

using System.IO;
using Newtonsoft.Json;

## Building and Using The GC Analysis API

In [ ]:
dotnet build -c Release "..\GC.Analysis.API"

In [8]:
// TODO: Ensure you are pointing to the right artifacts folder.
#r "..\..\..\..\..\artifacts\bin\GC.Analysis.API\Release\net6.0\GC.Analysis.API.dll"

using GC.Analysis.API;

## Data Acquisition

The next few cells detail how to retrieve the data from a base path. The run name below is the name of the folder generated from running the ``aspnetbenchmarks`` command from the GC.Infrastructure API. 

In [14]:
// The LoadInfo class consists of all the pertinent fields needed to represent both the result from a particular benchmark
// as well as the the comparison between two runs where the Data2 represents the GCProcessData of the comparand.
public sealed class LoadInfo
{
    public double MaxWorkingSetMB {get;set;} = double.NaN;
    public double P99WorkingSetMB {get;set;} = double.NaN;
    public double P95WorkingSetMB {get;set;} = double.NaN;
    public double P90WorkingSetMB {get;set;} = double.NaN;
    public double P75WorkingSetMB {get;set;} = double.NaN;
    public double P50WorkingSetMB {get;set;} = double.NaN;

    public double MaxPrivateMemoryMB {get;set;} = double.NaN;
    public double P99PrivateMemoryMB {get;set;} = double.NaN;
    public double P95PrivateMemoryMB {get;set;} = double.NaN;
    public double P90PrivateMemoryMB {get;set;} = double.NaN;
    public double P75PrivateMemoryMB {get;set;} = double.NaN;
    public double P50PrivateMemoryMB {get;set;} = double.NaN;

    public double Latency50thMS {get; set;} = double.NaN;
    public double Latency75thMS {get; set;} = double.NaN;
    public double Latency90thMS {get; set;} = double.NaN;
    public double Latency99thMS {get; set;} = double.NaN;
    public double MeanLatencyMS {get; set;} = double.NaN;
    public int ProcessId {get;set;}
    public double RequestsPerMSec {get; set;} = double.NaN;
    public string Run {get; set;}
    public GCProcessData Data {get;set;}
    public GCProcessData? Data2 {get;set;}
    public string CommandLine {get;set;}
    public double NumberOfHeapCountSwitches {get;set;} = 0;
    public string Benchmark {get; set;}
    public string Id {get; set;}
    public double TotalSuspensionTimeMSec {get;set;} = double.NaN;
    public double PercentPauseTimeInGC {get; set;} = double.NaN;
    public double PercentTimeInGC {get; set;} = double.NaN;
    public double MeanHeapSizeBeforeMB {get; set;} = double.NaN;
    public double MaxHeapSizeMB {get; set;} = double.NaN;
    public double TotalAllocationsMB {get;set;} = double.NaN;
    public double GCScore {get;set;} = double.NaN;
    public string TracePath {get; set;}
    public string ProcessName {get;set;}
}

public class BenchmarkVolatilityData
{
    public string Benchmark {get; set;}
    public double MaxWorkingSetMB {get;set;} = double.NaN;
    public double P99WorkingSetMB {get;set;} = double.NaN;
    public double P95WorkingSetMB {get;set;} = double.NaN;
    public double P90WorkingSetMB {get;set;} = double.NaN;
    public double P75WorkingSetMB {get;set;} = double.NaN;
    public double P50WorkingSetMB {get;set;} = double.NaN;

    public double MaxPrivateMemoryMB {get;set;} = double.NaN;
    public double P99PrivateMemoryMB {get;set;} = double.NaN;
    public double P95PrivateMemoryMB {get;set;} = double.NaN;
    public double P90PrivateMemoryMB {get;set;} = double.NaN;
    public double P75PrivateMemoryMB {get;set;} = double.NaN;
    public double P50PrivateMemoryMB {get;set;} = double.NaN;

    public double RequestsPerMSec {get;set;} = double.NaN;
    public double MeanLatencyMS {get; set;} = double.NaN;
    public double Latency50thMS {get; set;} = double.NaN;
    public double Latency75thMS {get; set;} = double.NaN;
    public double Latency90thMS {get; set;} = double.NaN;
    public double Latency99thMS {get; set;} = double.NaN;
    public double HeapCount { get; set; }= double.NaN;
}

In [18]:
// The DataManager is responsible for parsing all the data from ASP.NET results from a basepath.
public class DataManager 
{
    private readonly Dictionary<string, Dictionary<string, LoadInfo>> _benchmarkToRunData = new();
    private readonly Dictionary<string, Dictionary<string, LoadInfo>> _runToBenchmarkData = new();
    private readonly Dictionary<string, LoadInfo> _data; 
    private readonly string _basePath;

    public DataManager(string basePath)
    {
        _basePath = basePath;
        _data = GetLoadInfoFromBasePath(basePath);
        foreach (var d in _data)
        {
            if (!_benchmarkToRunData.TryGetValue(d.Value.Benchmark, out var runData))
            {
                _benchmarkToRunData[d.Value.Benchmark] = runData = new();
            }
            runData[d.Value.Run] = d.Value;

            if (!_runToBenchmarkData.TryGetValue(d.Value.Run, out var benchmarkData))
            {
                _runToBenchmarkData[d.Value.Run] = benchmarkData = new();
            }

            benchmarkData[d.Value.Benchmark] = d.Value;
        }
    }

    public static double DeltaPercent (double baseline, double comparand) => Math.Round((comparand - baseline) / baseline * 100, 2);

    public LoadInfo GetComparison(LoadInfo baseline, LoadInfo comparand)
    {
        return new LoadInfo
        {
            MaxWorkingSetMB    = DeltaPercent(baseline.MaxWorkingSetMB, comparand.MaxWorkingSetMB),
            P99WorkingSetMB = DeltaPercent(baseline.P99WorkingSetMB, comparand.P99WorkingSetMB),
            P95WorkingSetMB    = DeltaPercent(baseline.P95WorkingSetMB, comparand.P95WorkingSetMB),
            P90WorkingSetMB    = DeltaPercent(baseline.P90WorkingSetMB, comparand.P90WorkingSetMB),
            P75WorkingSetMB    = DeltaPercent(baseline.P75WorkingSetMB, comparand.P75WorkingSetMB),
            P50WorkingSetMB    = DeltaPercent(baseline.P50WorkingSetMB, comparand.P50WorkingSetMB),

            MaxPrivateMemoryMB = DeltaPercent(baseline.MaxPrivateMemoryMB, comparand.MaxPrivateMemoryMB),
            P99PrivateMemoryMB = DeltaPercent(baseline.P99PrivateMemoryMB, comparand.P99PrivateMemoryMB),
            P95PrivateMemoryMB    = DeltaPercent(baseline.P95PrivateMemoryMB, comparand.P95PrivateMemoryMB),
            P90PrivateMemoryMB    = DeltaPercent(baseline.P90PrivateMemoryMB, comparand.P90PrivateMemoryMB),
            P75PrivateMemoryMB    = DeltaPercent(baseline.P75PrivateMemoryMB, comparand.P75PrivateMemoryMB),
            P50PrivateMemoryMB    = DeltaPercent(baseline.P50PrivateMemoryMB, comparand.P50PrivateMemoryMB),
            
            Latency50thMS   = DeltaPercent(baseline.Latency50thMS, comparand.Latency50thMS),
            Latency75thMS   = DeltaPercent(baseline.Latency75thMS, comparand.Latency75thMS),
            Latency90thMS   = DeltaPercent(baseline.Latency90thMS, comparand.Latency90thMS), 
            Latency99thMS   = DeltaPercent(baseline.Latency99thMS, comparand.Latency99thMS),  
            MeanLatencyMS   = DeltaPercent(baseline.MeanLatencyMS, comparand.MeanLatencyMS),
            RequestsPerMSec = DeltaPercent(baseline.RequestsPerMSec, comparand.RequestsPerMSec),
            GCScore         = DeltaPercent(baseline.GCScore, comparand.GCScore),
            NumberOfHeapCountSwitches = DeltaPercent(baseline.NumberOfHeapCountSwitches, comparand.NumberOfHeapCountSwitches),
            Data = baseline.Data,
            Data2 = comparand.Data,
            Run = $"{baseline.Run} vs. {comparand.Run}",
            Benchmark = baseline.Benchmark,
            Id = $"{baseline.Run} vs. {comparand.Run} for {baseline.Benchmark}"
        };
    }

    public void SummarizeResults(Dictionary<string, LoadInfo> info = null)
    {
        if (info == null)
        {
            info = _data;
        }

        Console.WriteLine("{0,10} | {1,35} | {2, 5} | {3, 5:0.00} | {4, 5} | {5, 5:0.00} | {6, 5} | {7, 5:0.00} | {8, 5} | {9, 5:0.00} | {10, 10:0.00} | {11, 10:0.00} | {12, 5:0.00} | {13, 10:0.00} | {14, 10:0.00} |", 
                        "run", "benchmark", "gen0", "pause", "gen1", "pause", "ngc2", "pause", "bgc", "pause", "allocMB", "alloc/gc", "pct", "peakMB", "meanMB");
        Console.WriteLine("{0}", new String('-', 174));

        foreach (var kvp in info)
        {
            List<TraceGC> gcs = kvp.Value?.Data?.GCs;
            if (gcs == null || gcs.Count == 0)
            {
                continue;
            }

            int[] gc_counts = new int[4];
            double[] gc_pauses = new double[4];
            for (int i = 0; i < gcs.Count; i++)
            {
                TraceGC gc = gcs[i];
                if (gc.Generation < 2)
                {
                    gc_counts[gc.Generation]++;
                    gc_pauses[gc.Generation] += gc.PauseDurationMSec;
                }
                else
                {
                    if (gc.Type == GCType.BackgroundGC)
                    {
                        gc_counts[3]++;
                        gc_pauses[3] += gc.PauseDurationMSec;
                    }
                    else
                    {
                        gc_counts[2]++;
                        gc_pauses[2] += gc.PauseDurationMSec;
                    }
                }
            }
            
            for (int i = 0; i < 4; i++)
            {
                if (gc_counts[i] > 0)
                {
                    gc_pauses[i] /= gc_counts[i];
                }
            }
            
            Console.WriteLine("{0,10} | {1,35} | {2, 5} | {3, 5:0.00} | {4, 5} | {5, 5:0.00} | {6, 5} | {7, 5:0.00} | {8, 5} | {9, 5:0.00} | {10, 10:0.00} | {11, 10:0.00} | {12, 5:0.00} | {13, 10:0.00} | {14, 10:0.00} |",
                kvp.Value.Run, kvp.Value.Benchmark, gc_counts[0], gc_pauses[0], gc_counts[1], gc_pauses[1], gc_counts[2], gc_pauses[2], gc_counts[3], gc_pauses[3],
                kvp.Value.Data.Stats.TotalAllocatedMB, (kvp.Value.Data.Stats.TotalAllocatedMB / gcs.Count), kvp.Value.Data.Stats.GetGCPauseTimePercentage(), kvp.Value.Data.Stats.MaxSizePeakMB, kvp.Value.Data.Stats.MeanSizePeakMB);
        }
    }

    public Dictionary<string, LoadInfo>? GetAllBenchmarksForRun(string run)
    {
        if (!_runToBenchmarkData.TryGetValue(run, out var benchmarksForRun))
        {
            Console.WriteLine($"No benchmarks found for run: {run}");
            return null;
        }

        return benchmarksForRun;
    }

    public void SaveBenchmarkData(string outputPath = "")
    {
        if (string.IsNullOrEmpty(outputPath))
        {
            outputPath = _basePath;
        }

        StringBuilder sb = new();
        sb.AppendLine($"Run,Benchmark,Max Working Set (MB), Max Private Memory (MB), Request/MSec, Mean Latency (MSec), Latency 50th Percentile MSec, Latency 75th Percentile MSec, Latency 90th Percentile MSec, Latency 99th Percentile MSec");
        foreach (var b in _data)
        {
            var val = b.Value; 
            sb.AppendLine($"{val.Run},{val.Benchmark},{val.MaxWorkingSetMB},{val.MaxPrivateMemoryMB},{val.RequestsPerMSec},{val.MeanLatencyMS},{val.Latency50thMS},{val.Latency75thMS},{val.Latency90thMS},{val.Latency99thMS}");
        }

        File.WriteAllText(Path.Combine(outputPath, "AllBenchmarks.csv"), sb.ToString());
    }

    public Dictionary<string, LoadInfo>? GetAllRunsForBenchmark(string benchmark)
    {
        if (!_benchmarkToRunData.TryGetValue(benchmark, out var runsForBenchmark))
        {
            Console.WriteLine($"No runs found for benchmark: {benchmark}");
            return null;
        }

        return runsForBenchmark;
    }

    public LoadInfo? GetBenchmarkData(string benchmark, string run)
    {
        if (!_benchmarkToRunData.TryGetValue(benchmark, out var runData))
        {
            Console.WriteLine($"Benchmark: {benchmark} not found!");
            return null;
        }

        if (!runData.TryGetValue(run, out var loadInfo))
        {
            Console.WriteLine($"Run: {run} not found!");
            return null;
        }

        return loadInfo;
    }

    public Dictionary<string, LoadInfo> Data => _data; 

    private Dictionary<string, LoadInfo> GetLoadInfoFromBasePath(string basePath)
    {
        Dictionary<string, LoadInfo> flatLoadMap = new();
        var files = Directory.GetFiles(basePath, "*.log", SearchOption.AllDirectories);

        foreach (var f in files)
        {
            if (f.Contains("build.log") || f.Contains("output.log") || f.Contains("_GCLog"))
            {
                continue;
            }

            LoadInfo info = new();

            string[] lines = File.ReadAllLines(f);
            int idxOfApplication = Int32.MaxValue;
            int idxOfLoad = Int32.MaxValue;
            int idx = 0;

            foreach (var line in lines)
            {
                string[] sp = line.Split("|", StringSplitOptions.TrimEntries);
                if (line.Contains("| application"))
                {
                    idxOfApplication = idx;
                }

                else if (line.Contains("| load"))
                {
                    idxOfLoad = idx;
                }

                else if (line.Contains("| Latency 50th"))
                {
                    info.Latency50thMS = double.Parse(sp[2]);
                }

                else if (line.Contains("| Latency 75th"))
                {
                    info.Latency75thMS = double.Parse(sp[2]);
                }

                else if (line.Contains("| Latency 90th"))
                {
                    info.Latency90thMS = double.Parse(sp[2]);
                }

                else if (line.Contains("| Latency 99th"))
                {
                    info.Latency99thMS = double.Parse(sp[2]);
                }

                else if (line.Contains("Requests/sec"))
                {
                    info.RequestsPerMSec = double.Parse(sp[2]) / 1000;
                }

                else if (line.Contains("Mean latency"))
                {
                    info.MeanLatencyMS = double.Parse(sp[2]);
                }

                else if (line.Contains("Max Working Set") && (idxOfApplication < idx && idx < idxOfLoad)) 
                {
                    info.MaxWorkingSetMB = double.Parse(sp[2]);
                }

                else if (line.Contains("Working Set P99") && (idxOfApplication < idx && idx < idxOfLoad)) 
                {
                    info.P99WorkingSetMB = double.Parse(sp[2]);
                }

                else if (line.Contains("Working Set P95") && (idxOfApplication < idx && idx < idxOfLoad)) 
                {
                    info.P95WorkingSetMB = double.Parse(sp[2]);
                }

                else if (line.Contains("Working Set P90") && (idxOfApplication < idx && idx < idxOfLoad)) 
                {
                    info.P90WorkingSetMB = double.Parse(sp[2]);
                }
                
                else if (line.Contains("Working Set P75") && (idxOfApplication < idx && idx < idxOfLoad)) 
                {
                    info.P75WorkingSetMB = double.Parse(sp[2]);
                }

                else if (line.Contains("Working Set P50") && (idxOfApplication < idx && idx < idxOfLoad)) 
                {
                    info.P50WorkingSetMB = double.Parse(sp[2]);
                }
                
                else if (line.Contains("Max Private Memory") && (idxOfApplication < idx && idx < idxOfLoad)) 
                {
                    info.MaxPrivateMemoryMB  = double.Parse(sp[2]);
                }

                else if (line.Contains("Private Memory P99") && (idxOfApplication < idx && idx < idxOfLoad)) 
                {
                    info.P99WorkingSetMB = double.Parse(sp[2]);
                }

                else if (line.Contains("Private Memory P95") && (idxOfApplication < idx && idx < idxOfLoad)) 
                {
                    info.P95WorkingSetMB = double.Parse(sp[2]);
                }

                else if (line.Contains("Private Memory P90") && (idxOfApplication < idx && idx < idxOfLoad)) 
                {
                    info.P90WorkingSetMB = double.Parse(sp[2]);
                }
                
                else if (line.Contains("Private Memory P75") && (idxOfApplication < idx && idx < idxOfLoad)) 
                {
                    info.P75WorkingSetMB = double.Parse(sp[2]);
                }

                else if (line.Contains("Private Memory P50") && (idxOfApplication < idx && idx < idxOfLoad)) 
                {
                    info.P50WorkingSetMB = double.Parse(sp[2]);
                }

                ++idx;
            }

            string[] split =  f.Replace(".5", "5").Split(".");
            string run = split[1];
            string benchmark = Path.GetFileName( split[0] ).Replace("_Windows", "").Replace("_Linux", "").Replace(".gc", "").Replace(".nettrace", "");

            string key = $"{run} | {benchmark}";
            info.Benchmark = benchmark;
            info.Run = run;
            info.Id = key;

            flatLoadMap[key] = info;
        }

        var traceFiles = Directory.GetFiles(basePath, "*.etl.zip", SearchOption.AllDirectories).ToList();
        var nettraceFiles = Directory.GetFiles(basePath, "*.nettrace", SearchOption.AllDirectories);
        traceFiles.AddRange(nettraceFiles);

        HashSet<string> pertinentProcesses = new HashSet<string>
        {
            "PlatformBenchmarks",
            "Benchmarks",
            "MapAction",
            "TodosApi",
            "BasicGrpc",
            "BasicMinimalApi",
        };

        Parallel.ForEach(traceFiles, (t) => {
            string[] sp = t.Split("\\");
            string benchmark = Path.GetFileNameWithoutExtension(sp[sp.Length - 1]).Replace("_Windows", "").Replace(".gc.etl", "").Replace("_Linux", "").Replace(".nettrace", "").Replace(".gc", "");
            string name = sp[sp.Length - 2].Replace(".5", "5");
            string key = $"{name} | {benchmark}";

            Analyzer analyzer = AnalyzerManager.GetAnalyzer(t);
            GCProcessData? data = null;

            if (t.Contains(".nettrace"))
            {
                data = analyzer.AllGCProcessData.First().Value.First();
            }

            else
            {
                foreach (var p in pertinentProcesses)
                {
                    data = analyzer.GetProcessGCData(p).FirstOrDefault();
                    if (data != null)
                    {
                        break;
                    }
                }
            }

            if (data == null)
            {
                Console.WriteLine($"The following key doesn't have the pertinent process {key} - {t}: {string.Join(" , ", analyzer.TraceLog.Processes.Select(p => p.Name))}");
            }

            else
            {
                lock (flatLoadMap)
                {
                    if (flatLoadMap.TryGetValue(key, out var f))
                    {
                        f.MeanHeapSizeBeforeMB = data.Stats.MeanSizePeakMB;
                        f.MaxHeapSizeMB = data.Stats.MaxSizePeakMB;
                        f.PercentTimeInGC = (data.GCs.Sum(gc => gc.PauseDurationMSec - gc.SuspendDurationMSec) / (data.Stats.ProcessDuration) ) * 100;
                        f.TracePath = data.Parent.TraceLogPath;
                        f.TotalAllocationsMB = data.Stats.TotalAllocatedMB;
                        f.CommandLine = data.CommandLine;
                        f.PercentPauseTimeInGC = data.Stats.GetGCPauseTimePercentage();
                        f.GCScore = (f.MaxHeapSizeMB / f.PercentPauseTimeInGC);
                        f.ProcessId = data.ProcessID;
                        f.Data = data;
                        f.ProcessName = data.ProcessName;
                        f.TotalSuspensionTimeMSec = data.GCs.Sum(gc => gc.SuspendDurationMSec);

                        for (int i = 0; i < data.GCs.Count - 1; i++)
                        {
                            if ( data.GCs[i].GlobalHeapHistory?.NumHeaps != data.GCs[i + 1].GlobalHeapHistory?.NumHeaps)
                            {
                                ++f.NumberOfHeapCountSwitches;
                            }
                        }

                    }

                    else
                    {
                        Console.WriteLine($"{key} not found - Check if the trace has any elements: {t}");
                    }
                }
            }
        });

        return flatLoadMap;
    }

    public Dictionary<string, LoadInfo> GetBenchmarkToComparison(string baselineRun, string comparandRun)
    {
        Dictionary<string, LoadInfo> comparisons = new();

        Dictionary<string, LoadInfo> baselineData = new();
        Dictionary<string, LoadInfo> comparandData = new();
        HashSet<string> allBenchmarks = new();

        foreach (var d in _data)
        {
            allBenchmarks.Add(d.Value.Benchmark);

            string run = d.Key.Split("|", StringSplitOptions.RemoveEmptyEntries | StringSplitOptions.TrimEntries)[0];

            if (string.CompareOrdinal(run, baselineRun) == 0 && !baselineData.TryGetValue(d.Key, out var baselineInfo))
            {
                baselineInfo = baselineData[d.Value.Benchmark] = d.Value;
            }

            else if (string.CompareOrdinal(run, comparandRun) == 0 && !comparandData.TryGetValue(d.Key, out var comparandInfo))
            {
                comparandInfo = comparandData[d.Value.Benchmark] = d.Value;
            }
        }

        foreach (var benchmark in allBenchmarks)
        {
            if (!baselineData.TryGetValue(benchmark, out var baselineBenchmarkInfo))
            {
                Console.WriteLine($"Benchmark: {benchmark} not found on the baseline: {baselineRun}");
                continue;
            }

            if (!comparandData.TryGetValue(benchmark, out var comparandBenchmarkInfo))
            {
                Console.WriteLine($"Benchmark: {benchmark} not found on the comparand: {comparandRun}");
                continue;
            }

            LoadInfo comparison = GetComparison(baselineBenchmarkInfo, comparandBenchmarkInfo);
            comparisons[benchmark] = comparison;
        }
        
        return comparisons;
    }

    private static double ComputeVolatility(List<double> data)
    {
        var max = data.Max();
        var min = data.Min();
        return Math.Round(((max - min) / min) * 100, 2);
    }

    public void SaveVolatilityData(List<string> namesOfBuilds, List<string> sortingCriteria = null)
    {
        // Build Parent -> < Build Name -> < Benchmark -> Data >>>
        Dictionary<string, Dictionary<string, Dictionary<string, LoadInfo>>> listOfData = new();

        foreach (var build in namesOfBuilds)
        {
            if (!listOfData.TryGetValue(build, out var b))
            {
                listOfData[build] = b = new();
            }

            foreach (var run in _runToBenchmarkData)
            {
                if (run.Key.Contains(build))
                {
                    b.Add(run.Key, run.Value);
                }
            }
        }

        // At this point all the data has been categorized.
        Dictionary<string, Dictionary<string, BenchmarkVolatilityData>> buildToBenchmarkVolatilityData = new();

        // Get the Volatility Data Per Build.
        foreach (var b in listOfData)
        {
            if (!buildToBenchmarkVolatilityData.TryGetValue(b.Key, out var volData))
            {
                buildToBenchmarkVolatilityData[b.Key] = volData = new();
            }

            foreach (var br in _benchmarkToRunData)
            {
                volData[br.Key] = new();
            }

            Dictionary<string, List<LoadInfo>> benchmarkToData = new();
            foreach (var run in b.Value)
            {
                foreach (var benchmark in run.Value)
                {
                    if (!benchmarkToData.TryGetValue(benchmark.Key, out var d))
                    {
                        benchmarkToData[benchmark.Key] = d = new();
                    }

                    d.Add(benchmark.Value);
                }
            }

            foreach (var benchmark in benchmarkToData)
            {
                volData[benchmark.Key] = new BenchmarkVolatilityData
                {
                    MaxWorkingSetMB = ComputeVolatility( benchmark.Value.Select(v => v.MaxWorkingSetMB) ),
                    P99WorkingSetMB = ComputeVolatility( benchmark.Value.Select(v => v.P99WorkingSetMB) ),
                    P95WorkingSetMB = ComputeVolatility( benchmark.Value.Select(v => v.P95WorkingSetMB) ),
                    P90WorkingSetMB = ComputeVolatility( benchmark.Value.Select(v => v.P90WorkingSetMB) ),
                    P75WorkingSetMB = ComputeVolatility( benchmark.Value.Select(v => v.P75WorkingSetMB) ),
                    P50WorkingSetMB = ComputeVolatility( benchmark.Value.Select(v => v.P50WorkingSetMB) ),

                    MaxPrivateMemoryMB = ComputeVolatility( benchmark.Value.Select(v => v.MaxPrivateMemoryMB) ),
                    P99PrivateMemoryMB = ComputeVolatility( benchmark.Value.Select(v => v.P99PrivateMemoryMB) ),
                    P95PrivateMemoryMB = ComputeVolatility( benchmark.Value.Select(v => v.P95PrivateMemoryMB) ),
                    P90PrivateMemoryMB = ComputeVolatility( benchmark.Value.Select(v => v.P90PrivateMemoryMB) ),
                    P75PrivateMemoryMB = ComputeVolatility( benchmark.Value.Select(v => v.P75PrivateMemoryMB) ),
                    P50PrivateMemoryMB = ComputeVolatility( benchmark.Value.Select(v => v.P50PrivateMemoryMB) ),

                    RequestsPerMSec = ComputeVolatility( benchmark.Value.Select(v => v.RequestsPerMSec) ),
                    MeanLatencyMS = ComputeVolatility( benchmark.Value.Select(v => v.MeanLatencyMS) ),
                    Latency50thMS = ComputeVolatility( benchmark.Value.Select(v => v.Latency50thMS) ),
                    Latency75thMS = ComputeVolatility( benchmark.Value.Select(v => v.Latency75thMS) ),
                    Latency90thMS = ComputeVolatility( benchmark.Value.Select(v => v.Latency90thMS) ),
                    Latency99thMS = ComputeVolatility( benchmark.Value.Select(v => v.Latency99thMS) ),
                    HeapCount     = ComputeVolatility( benchmark.Value.Select(v => (double)v.NumberOfHeapCountSwitches )),
                    Benchmark     = benchmark.Key,
                };
            }
        }
       
        Dictionary<string, List<BenchmarkVolatilityData>> sortedPerBuildVolatility = new();

        string DisplayDetailsForABenchmark(BenchmarkVolatilityData val) =>
            $"{val.Benchmark},{val.MaxWorkingSetMB},{val.MaxPrivateMemoryMB},{val.RequestsPerMSec},{val.MeanLatencyMS},{val.Latency50thMS},{val.Latency75thMS},{val.Latency90thMS},{val.Latency99thMS},{val.HeapCount}";
        if (sortingCriteria == null)
        {
            sortingCriteria = new() { nameof(LoadInfo.MaxPrivateMemoryMB) };
        }

        foreach (var s in sortingCriteria)
        {
            Func<KeyValuePair<string, BenchmarkVolatilityData>, double> sortingFunctor = null;
            Func<BenchmarkVolatilityData, double> selectionFunctor = null;

            switch (s)
            {
                case nameof(BenchmarkVolatilityData.MaxWorkingSetMB):
                    sortingFunctor = (data) => data.Value.MaxWorkingSetMB;
                    selectionFunctor = (data) => data.MaxWorkingSetMB;
                    break;
                case nameof(BenchmarkVolatilityData.MaxPrivateMemoryMB):
                    sortingFunctor = (data) => data.Value.MaxPrivateMemoryMB;
                    selectionFunctor = (data) => data.MaxPrivateMemoryMB;
                    break;
                case nameof(BenchmarkVolatilityData.RequestsPerMSec):
                    sortingFunctor = (data) => data.Value.RequestsPerMSec;
                    selectionFunctor = (data) => data.RequestsPerMSec;
                    break;
                case nameof(BenchmarkVolatilityData.MeanLatencyMS):
                    sortingFunctor = (data) => data.Value.MeanLatencyMS;
                    selectionFunctor = (data) => data.MeanLatencyMS;
                    break;
                case nameof(BenchmarkVolatilityData.Latency50thMS):
                    sortingFunctor = (data) => data.Value.Latency50thMS;
                    selectionFunctor = (data) => data.Latency50thMS;
                    break;
                case nameof(BenchmarkVolatilityData.Latency75thMS):
                    sortingFunctor = (data) => data.Value.Latency75thMS;
                    selectionFunctor = (data) => data.Latency75thMS;
                    break;
                case nameof(BenchmarkVolatilityData.Latency90thMS):
                    sortingFunctor = (data) => data.Value.Latency90thMS;
                    selectionFunctor = (data) => data.Latency90thMS;
                    break;
                case nameof(BenchmarkVolatilityData.Latency99thMS):
                    sortingFunctor = (data) => data.Value.Latency99thMS;
                    selectionFunctor = (data) => data.Latency99thMS;
                    break;
                default:
                    sortingFunctor = (data) => data.Value.MaxPrivateMemoryMB;
                    selectionFunctor = (data) => data.MaxPrivateMemoryMB;
                    break;
            }

            foreach (var b in buildToBenchmarkVolatilityData)
            {
                sortedPerBuildVolatility[b.Key] = b.Value.OrderByDescending(sortingFunctor).Select(k => k.Value).ToList();
            }

            // Create CSV.
            StringBuilder top = new();

            // Iterate over each of the runs.
            const int singleBuildColumnSize = 10;
            int numberOfBuilds = buildToBenchmarkVolatilityData.Count;
            string columnHeader = "Benchmark Name,MaxWorkingSetMB,MaxPrivateMemoryMB,RequestsPerMSec,MeanLatencyMS,Latency50thMS,Latency75thMS,Latency90thMS,Latency99thMS,HeapCount";

            // Assumption: the same benchmarks are present for all runs.
            int totalCountOfBenchmarks = buildToBenchmarkVolatilityData.First().Value.Count;

            string first = string.Join(",", namesOfBuilds.Select(build => build + string.Join("", Enumerable.Repeat(",", singleBuildColumnSize))));
            string second = string.Join(",,", Enumerable.Repeat(columnHeader, numberOfBuilds));

            top.AppendLine(first);
            top.AppendLine(second);

            for (int benchmarkIdx = 0; benchmarkIdx < totalCountOfBenchmarks; benchmarkIdx++)
            {
                top.AppendLine(string.Join(",,", namesOfBuilds.Select(buildName => DisplayDetailsForABenchmark(sortedPerBuildVolatility[buildName][benchmarkIdx]))));
            }

            File.WriteAllText(Path.Combine(_basePath, $"Volatility_{s}.csv"), top.ToString());

            // Chart the sorted % Vol Results.

            List<Scatter> scatters = new();

            var layout = new Layout.Layout
            {
                xaxis = new Xaxis { title = "Benchmark Name" },
                yaxis = new Yaxis { title = "Metric Volatility Score" },
                width = 1500,
                title = $"Volatility Scores Sorted by {s} for {sortedPerBuildVolatility.First().Key}"
            };

            foreach (var b in sortedPerBuildVolatility)
            {
                var scatter = new Scatter
                {
                    x = b.Value.Select(s => s.Benchmark),
                    y = b.Value.Select(selectionFunctor),
                    mode = "markers",
                    name = b.Key,
                };

                scatters.Add(scatter);
            }

            Chart.Plot(scatters, layout).Display();

            scatters.Clear();
            layout = new Layout.Layout
            {
                xaxis = new Xaxis { title = "Volatility Index" },
                yaxis = new Yaxis { title = "Metric Volatility Score" },
                width = 1500,
                title = $"Volatility Index Sorted by {s} for {sortedPerBuildVolatility.First().Key}"
            };

            foreach (var b in sortedPerBuildVolatility)
            {
                var sorted = b.Value.OrderByDescending(selectionFunctor);
                var scatter = new Scatter
                {
                    x = Enumerable.Range(0, sorted.Count()),
                    y = sorted.Select(selectionFunctor),
                    mode = "markers",
                    name = b.Key,
                    text = sorted.Select(ss => ss.Benchmark),
                };

                scatters.Add(scatter);
            }
            
            Chart.Plot(scatters, layout).Display();
        }
    }

    public void SaveDifferences(string baseline, string comparand, List<string> sortingCriteria = null)
    {
        // This function assumes the runs are all in:
        // {build}_{iteration} form.
        // Else, it will except.

        // Iteration -> LoadInfos
        Dictionary<int, List<LoadInfo>> iterationData = new();

        // Get the max iteration.
        int maxIteration = -1;
        foreach (var run in _runToBenchmarkData)
        {
            string runName = run.Key;
            string[] split = run.Key.Split("_");
            Debug.Assert(split.Length == 2);
            string build = split[0];
            string iterationAsString = split[1];
            int iteration = Convert.ToInt32(iterationAsString);
            maxIteration = System.Math.Max(iteration, maxIteration);
        }

        // Compute Average Diff
        // Build to Benchmark -> Data
        Dictionary<string, Dictionary<string, LoadInfo>> averageData = new();

        for (int i = 0; i <= maxIteration; i++)
        {
            string baselineIteration  = baseline  + "_" + i.ToString();
            string comparandIteration = comparand + "_" + i.ToString();

            Dictionary<string, LoadInfo> baselineIterationRuns  = _runToBenchmarkData[baselineIteration];
            Dictionary<string, LoadInfo> comparandIterationRuns = _runToBenchmarkData[comparandIteration];

            foreach (var b in baselineIterationRuns)
            {
                if (!iterationData.TryGetValue(i, out var benchmarks))
                {
                    iterationData[i] = benchmarks = new();
                }

                benchmarks.Add(GetComparison(baselineIterationRuns[b.Key], comparandIterationRuns[b.Key]));
            }

            if (!averageData.TryGetValue(baseline, out var bVal))
            {
                averageData[baseline] = bVal = new();
                foreach (var benchmark in baselineIterationRuns)
                {
                    bVal[benchmark.Key] = new LoadInfo
                    {
                        Benchmark = benchmark.Key,
                        MaxWorkingSetMB    = benchmark.Value.MaxWorkingSetMB,
                        MaxPrivateMemoryMB = benchmark.Value.MaxPrivateMemoryMB,
                        RequestsPerMSec = benchmark.Value.RequestsPerMSec,
                        MeanLatencyMS   = benchmark.Value.MeanLatencyMS,
                        Latency50thMS   = benchmark.Value.Latency50thMS, 
                        Latency75thMS   = benchmark.Value.Latency75thMS,
                        Latency90thMS   = benchmark.Value.Latency90thMS,
                        Latency99thMS   = benchmark.Value.Latency99thMS,
                        NumberOfHeapCountSwitches = benchmark.Value.NumberOfHeapCountSwitches,
                    };
                }
            }

            else
            {
                foreach (var benchmark in baselineIterationRuns)
                {
                    var data = bVal[benchmark.Key];
                    data.Benchmark = benchmark.Key;
                    data.MaxWorkingSetMB    += benchmark.Value.MaxWorkingSetMB;
                    data.MaxPrivateMemoryMB += benchmark.Value.MaxPrivateMemoryMB;
                    data.RequestsPerMSec += benchmark.Value.RequestsPerMSec;
                    data.MeanLatencyMS   += benchmark.Value.MeanLatencyMS;
                    data.Latency50thMS   += benchmark.Value.Latency50thMS; 
                    data.Latency75thMS   += benchmark.Value.Latency75thMS;
                    data.Latency90thMS   += benchmark.Value.Latency90thMS;
                    data.Latency99thMS   += benchmark.Value.Latency99thMS;
                    data.NumberOfHeapCountSwitches += benchmark.Value.NumberOfHeapCountSwitches;
                }
            }

            if (!averageData.TryGetValue(comparand, out var cVal))
            {
                averageData[comparand] = cVal = new();
                foreach (var benchmark in comparandIterationRuns)
                {
                    cVal[benchmark.Key] = new LoadInfo
                    {
                        Benchmark = benchmark.Key,
                        MaxWorkingSetMB    = benchmark.Value.MaxWorkingSetMB,
                        MaxPrivateMemoryMB = benchmark.Value.MaxPrivateMemoryMB,
                        RequestsPerMSec = benchmark.Value.RequestsPerMSec,
                        MeanLatencyMS   = benchmark.Value.MeanLatencyMS,
                        Latency50thMS   = benchmark.Value.Latency50thMS, 
                        Latency75thMS   = benchmark.Value.Latency75thMS,
                        Latency90thMS   = benchmark.Value.Latency90thMS,
                        Latency99thMS   = benchmark.Value.Latency99thMS,
                        NumberOfHeapCountSwitches = benchmark.Value.NumberOfHeapCountSwitches,
                    };
                }
            }

            else
            {
                foreach (var benchmark in comparandIterationRuns)
                {
                    var data = cVal[benchmark.Key];
                    data.Benchmark = benchmark.Key;
                    data.MaxWorkingSetMB    += benchmark.Value.MaxWorkingSetMB;
                    data.MaxPrivateMemoryMB += benchmark.Value.MaxPrivateMemoryMB;
                    data.RequestsPerMSec += benchmark.Value.RequestsPerMSec;
                    data.MeanLatencyMS   += benchmark.Value.MeanLatencyMS;
                    data.Latency50thMS   += benchmark.Value.Latency50thMS; 
                    data.Latency75thMS   += benchmark.Value.Latency75thMS;
                    data.Latency90thMS   += benchmark.Value.Latency90thMS;
                    data.Latency99thMS   += benchmark.Value.Latency99thMS;
                    data.NumberOfHeapCountSwitches += benchmark.Value.NumberOfHeapCountSwitches;
                }
            }
        }

        foreach (var benchmark in _benchmarkToRunData)
        {
            foreach (var build in averageData)
            {
                var data = build.Value[benchmark.Key];
                data.Benchmark = benchmark.Key;
                data.MaxWorkingSetMB    /= (maxIteration + 1); 
                data.MaxPrivateMemoryMB /=  (maxIteration + 1);
                data.RequestsPerMSec /=  (maxIteration + 1);
                data.MeanLatencyMS   /= (maxIteration + 1);
                data.Latency50thMS   /= (maxIteration + 1);
                data.Latency75thMS   /= (maxIteration + 1);
                data.Latency90thMS   /= (maxIteration + 1);
                data.Latency99thMS   /= (maxIteration + 1);
                data.NumberOfHeapCountSwitches /= (maxIteration + 1);
            }
        }

        string DisplayDetailsForABenchmark(LoadInfo val) =>
            $"{val.Benchmark},{val.MaxWorkingSetMB},{val.MaxPrivateMemoryMB},{val.RequestsPerMSec},{val.MeanLatencyMS},{val.Latency50thMS},{val.Latency75thMS},{val.Latency90thMS},{val.Latency99thMS},{val.NumberOfHeapCountSwitches}";

        if (sortingCriteria == null)
        {
            sortingCriteria = new() { nameof(LoadInfo.MaxPrivateMemoryMB) };
        }

        foreach (var s in sortingCriteria)
        {
            Func<LoadInfo, double> sortingFunctor = null;
            Func<KeyValuePair<string, LoadInfo>, double> selectionFunctor = null;

            switch (s)
            {
                case nameof(LoadInfo.MaxWorkingSetMB):
                    sortingFunctor = (data) => data.MaxWorkingSetMB;
                    selectionFunctor = (data) => data.Value.MaxWorkingSetMB;
                    break;
                case nameof(LoadInfo.MaxPrivateMemoryMB):
                    sortingFunctor = (data) => data.MaxPrivateMemoryMB;
                    selectionFunctor = (data) => data.Value.MaxPrivateMemoryMB;
                    break;
                case nameof(LoadInfo.RequestsPerMSec):
                    sortingFunctor = (data) => data.RequestsPerMSec;
                    selectionFunctor = (data) => data.Value.RequestsPerMSec;
                    break;
                case nameof(LoadInfo.MeanLatencyMS):
                    sortingFunctor = (data) => data.MeanLatencyMS;
                    selectionFunctor = (data) => data.Value.MeanLatencyMS;
                    break;
                case nameof(LoadInfo.Latency50thMS):
                    sortingFunctor = (data) => data.Latency50thMS;
                    selectionFunctor = (data) => data.Value.Latency50thMS;
                    break;
                case nameof(LoadInfo.Latency75thMS):
                    sortingFunctor = (data) => data.Latency75thMS;
                    selectionFunctor = (data) => data.Value.Latency75thMS;
                    break;
                case nameof(LoadInfo.Latency90thMS):
                    sortingFunctor = (data) => data.Latency90thMS;
                    selectionFunctor = (data) => data.Value.Latency90thMS;
                    break;
                case nameof(LoadInfo.Latency99thMS):
                    sortingFunctor = (data) => data.Latency99thMS;
                    selectionFunctor = (data) => data.Value.Latency99thMS;
                    break;
                case nameof(LoadInfo.NumberOfHeapCountSwitches):
                    sortingFunctor = (data) => data.NumberOfHeapCountSwitches;
                    selectionFunctor = (data) => data.Value.NumberOfHeapCountSwitches;
                    break;
                default:
                    sortingFunctor = (data) => data.MaxPrivateMemoryMB;
                    selectionFunctor = (data) => data.Value.MaxPrivateMemoryMB;
                    break;
            }

            List<List<LoadInfo>> sortedLoadInfo = new(); 
            foreach (var iteration in iterationData)
            {
                sortedLoadInfo.Add(iteration.Value.OrderByDescending(sortingFunctor).ToList());
            }

            List<LoadInfo> sortedAverages = new();

            foreach (var benchmark in averageData[baseline])
            {
                LoadInfo baselineInfo   = benchmark.Value;
                LoadInfo comparandInfo  = averageData[comparand][benchmark.Key];
                LoadInfo comparisonInfo = GetComparison(baselineInfo, comparandInfo);
                sortedAverages.Add(comparisonInfo);
            }
            sortedAverages = sortedAverages.OrderByDescending(sortingFunctor).ToList();

            // Create CSV.
            StringBuilder top = new();

            // Iterate over each of the runs.
            const int singleBuildColumnSize = 11;
            int numberOfIterations = maxIteration + 1;
            string columnHeader = "Benchmark Name,WorkingSetMB,PrivateMemoryMB,RequestsPerMSec,MeanLatencyMS,Latency50thMS,Latency75thMS,Latency90thMS,Latency99thMS,HeapCount";

            int totalCountOfBenchmarks = sortedLoadInfo.First().Count;

            string first = string.Join("", Enumerable.Range(0, numberOfIterations).Select(build => build + string.Join("", Enumerable.Repeat(",", singleBuildColumnSize))));
            string second = string.Join(",,", Enumerable.Repeat(columnHeader, numberOfIterations));

            // Add the average diff.
            first  += "Average Diff %" + string.Join("", Enumerable.Repeat(",", singleBuildColumnSize));
            second += ",," + string.Join(",,", columnHeader);

            top.AppendLine(first);
            top.AppendLine(second);

            for (int benchmarkIdx = 0; benchmarkIdx < totalCountOfBenchmarks; benchmarkIdx++)
            {
                string benchmarkData = string.Join(",,", Enumerable.Range(0, numberOfIterations).Select(iteration => DisplayDetailsForABenchmark(sortedLoadInfo[iteration][benchmarkIdx])));
                benchmarkData += $",,{DisplayDetailsForABenchmark(sortedAverages[benchmarkIdx])}";

                top.AppendLine(benchmarkData);
            }

            File.WriteAllText(Path.Combine(_basePath, $"Difference_{s}.csv"), top.ToString());

            var layout = new Layout.Layout
            {
                xaxis = new Xaxis { title = "Benchmark Name" },
                yaxis = new Yaxis { title = $"{s}" },
                width = 1500,
                title = $"Raw values of {s} for Runs"
            };

            List<Scatter> scatters = new();

            const int baseColor = 150;

            for (int iterationIdx = 0; iterationIdx <= maxIteration; iterationIdx++)
            {
                string baselineIteration  = baseline  + "_" + iterationIdx.ToString();
                string comparandIteration = comparand + "_" + iterationIdx.ToString();

                Dictionary<string, LoadInfo> baselineData  = _runToBenchmarkData[baselineIteration];
                Dictionary<string, LoadInfo> comparandData = _runToBenchmarkData[comparandIteration];

                if (iterationIdx == 0)
                {
                    var sortedBaseline = baselineData.Values.OrderByDescending(sortingFunctor);
                    baselineData = sortedBaseline.ToDictionary(d => d.Benchmark);
                }

                Scatter baselineScatter = new()
                {
                    x = baselineData.Select(b => b.Key),
                    y = baselineData.Select(selectionFunctor),
                    name = $"{baselineIteration} - {s}",
                    mode = "markers",
                    marker = new Marker { color = $"rgb({baseColor + iterationIdx * 50}, 0, 0)" } 
                };

                Scatter comparandScatter = new()
                {
                    x = comparandData.Select(b => b.Key),
                    y = comparandData.Select(selectionFunctor),
                    name = $"{comparandIteration} - {s}",
                    mode = "markers",
                    marker = new Marker { color = $"rgb(0, 0, {baseColor + iterationIdx * 50})" } 
                };

                scatters.Add(baselineScatter);
                scatters.Add(comparandScatter);
            }

            Chart.Plot(scatters, layout).Display();
        }
    }
}

In [ ]:
string basePath = @"C:\Traces\GCTraces\DATAS_5_Fixed";
var dataManager = new DataManager(basePath);

In [ ]:
dataManager.SummarizeResults()

In [ ]:
dataManager.SaveVolatilityData(new List<string> { "baseline", "run" }, new List<string> { nameof(LoadInfo.PrivateMemoryMB), nameof(LoadInfo.RequestsPerMSec )});

In [ ]:
dataManager.SaveDifferences("baseline", "run", new List<string> { nameof(LoadInfo.PrivateMemoryMB), nameof(LoadInfo.RequestsPerMSec) });

In [ ]:
dataManager.Data

### Using the DataManager

The following cells demonstrates how to make use of the ``DataManager``. 

In [ ]:
// The name of the run from the yaml file for which the ASP.NET run is created for.
string runName = "run";

Dictionary<string, LoadInfo> run = dataManager.GetAllBenchmarksForRun(runName);
dataManager.Data.Display();
List<KeyValuePair<string, LoadInfo>> runsWithGCData = dataManager.GetAllBenchmarksForRun(runName).Where(gc => gc.Value.Data != null);

In [ ]:
string benchmarkName = "Name of the specific benchmark";
LoadInfo benchmarkData = dataManager.GetBenchmarkData(benchmark: benchmarkName, run: runName);
benchmarkData.Id

In [ ]:
Dictionary<string, LoadInfo> allRunsForBenchmark = dataManager.GetAllRunsForBenchmark(benchmark: benchmarkName);
allRunsForBenchmark.Keys

#### Saving The Benchmark Results

The following call will persist a flat list of all the results.

In [ ]:
dataManager.SaveBenchmarkData()

## Build to Build Comparison and Volatility Analysis

In [ ]:
var run1_vs_run2 = dataManager.GetBenchmarkToComparison("datas_2", "datas_3");

In [25]:
static bool IsNotInvalidDouble(double val) => 
    !double.IsNaN(val) && 
    !double.IsInfinity(val) && 
    !double.IsPositiveInfinity(val) && 
    !double.IsNegativeInfinity(val);

public class SummaryTable
{
    public SummaryTable(Dictionary<string, Dictionary<string, LoadInfo>> comparisons)
    {
        Comparisons = comparisons;
    }

    private string GenerateSummaryForComparison(string comparisonKey, Dictionary<string, LoadInfo> comparison)
    {
        double averageWorkingSet = comparison.Where(a => IsNotInvalidDouble(a.Value.MaxWorkingSetMB)).Average(a => a.Value.MaxWorkingSetMB);
        double privateMemory = comparison.Where(a => IsNotInvalidDouble(a.Value.MaxPrivateMemoryMB)).Average(a => a.Value.MaxPrivateMemoryMB);
        double throughput = comparison.Where(a => IsNotInvalidDouble(a.Value.RequestsPerMSec)).Average(a => a.Value.RequestsPerMSec);
        double meanLatency = comparison.Where(a => IsNotInvalidDouble(a.Value.MeanLatencyMS)).Average(a => a.Value.MeanLatencyMS);

        double p50Latency = comparison.Where(a => IsNotInvalidDouble(a.Value.Latency50thMS)).Average(a => a.Value.Latency50thMS);
        double p75Latency = comparison.Where(a => IsNotInvalidDouble(a.Value.Latency75thMS)).Average(a => a.Value.Latency75thMS);
        double p90Latency = comparison.Where(a => IsNotInvalidDouble(a.Value.Latency90thMS)).Average(a => a.Value.Latency90thMS);
        double p99Latency = comparison.Where(a => IsNotInvalidDouble(a.Value.Latency99thMS)).Average(a => a.Value.Latency99thMS);

        return $"{comparisonKey},{averageWorkingSet},{privateMemory},{throughput},{meanLatency},{p50Latency},{p75Latency},{p90Latency},{p99Latency}";
    }

    public string GenerateSummaryForComparisons()
    {
        StringBuilder sb = new();
        sb.AppendLine("Build to Build,Average Max Working Set (MB) %, Average Max Private Memory (MB) %, Average Request/MSec %, Average Mean Latency (MSec), Average P50 Latency (MSec) %, Average P75 Latency (MSec) %, Average P90 Latency (MSec) %, Average P99 Latency (MSec) %");
        foreach (var comparison in Comparisons)
        {
            sb.AppendLine(GenerateSummaryForComparison(comparison.Key, comparison.Value));
        }

        return sb.ToString();
    }

    private int GetCountOfRegressions(List<double> selected, double thresholdPercentage, bool lessIsBetter = true)
    {
        // If throughput, less is worse => threshold <= -5%.
        var comparison = selected.Where(d => IsNotInvalidDouble(d) && ( (lessIsBetter) ? (d >= thresholdPercentage) : (d <= -thresholdPercentage)));
        return comparison.Count;
    }

    private int GetCountOfAbsRegressions(List<double> selected, double thresholdPercentage)
    {
        var comparison = selected.Where(d => IsNotInvalidDouble(d) && Math.Abs(d) >= thresholdPercentage);
        return comparison.Count;
    }

    // # of benchmarks with throughput regressed by >= 5% and 10%
    private string GenerateRegressionSummary(string comparisonKey, Dictionary<string, LoadInfo> comparison)
    {
        List<double> workingSet  = comparison.Select(c => c.Value.MaxWorkingSetMB);
        int workingSetCountGT_5  = GetCountOfRegressions(workingSet, 5);
        int workingSetCountGT_10 = GetCountOfRegressions(workingSet, 10);

        List<double> privateMemory  = comparison.Select(c => c.Value.MaxPrivateMemoryMB);
        int privateMemoryCountGT_5  = GetCountOfRegressions(privateMemory, 5);
        int privateMemoryCountGT_10 = GetCountOfRegressions(privateMemory, 10);

        List<double> throughput  = comparison.Select(a => a.Value.RequestsPerMSec);
        int throughputCountGT_5  = GetCountOfRegressions(throughput, 5, false);
        int throughputCountGT_10 = GetCountOfRegressions(throughput, 10, false);

        List<double> meanLatency  = comparison.Select(a => a.Value.MeanLatencyMS);
        int meanLatencyCountGT_5  = GetCountOfRegressions(meanLatency, 5);
        int meanLatencyCountGT_10 = GetCountOfRegressions(meanLatency, 10);

        List<double> p50Latency  = comparison.Select(a => a.Value.Latency50thMS);
        int p50LatencyCountGT_5  = GetCountOfRegressions(p50Latency, 5);
        int p50LatencyCountGT_10 = GetCountOfRegressions(p50Latency, 10);

        List<double> p75Latency  = comparison.Select(a => a.Value.Latency75thMS);
        int p75LatencyCountGT_5  = GetCountOfRegressions(p75Latency, 5);
        int p75LatencyCountGT_10 = GetCountOfRegressions(p75Latency, 10);

        List<double> p90Latency  = comparison.Select(a => a.Value.Latency90thMS);
        int p90LatencyCountGT_5  = GetCountOfRegressions(p90Latency, 5);
        int p90LatencyCountGT_10 = GetCountOfRegressions(p90Latency, 10);
        
        List<double> p99Latency  = comparison.Select(a => a.Value.Latency99thMS);
        int p99LatencyCountGT_5  = GetCountOfRegressions(p99Latency, 5);
        int p99LatencyCountGT_10 = GetCountOfRegressions(p99Latency, 10);

        return $"{comparisonKey},{workingSetCountGT_5},{workingSetCountGT_10},{privateMemoryCountGT_5},{privateMemoryCountGT_10},{throughputCountGT_5},{throughputCountGT_10},{meanLatencyCountGT_5},{meanLatencyCountGT_10},{p50LatencyCountGT_5},{p50LatencyCountGT_10},{p75LatencyCountGT_5},{p75LatencyCountGT_10},{p90LatencyCountGT_5},{p90LatencyCountGT_10},{p99LatencyCountGT_5},{p99LatencyCountGT_10}";
    }

    public string GenerateRegressionSummaryForComparisons()
    {
        StringBuilder sb = new();
        sb.AppendLine("Build to Build,Reg. Count - Working Set (MB),Large Reg. Count - Working Set (MB),Reg. Count - Max Private Memory (MB),Large Reg. Count - Max Private Memory (MB),Reg. Count - Throughput, Large Reg. Count - Throughput,Reg. Count - Mean Latency,Large Reg. Count - Mean Latency,Reg. Count - P50 Latency, Large Reg. Count - P50 Latency, Reg. Count - P75 Latency, Large Reg. Count - P75 Latency,Reg. Count - P90 Latency, Large Reg. Count - P90 Latency,Reg. Count - P99 Latency, Large Reg. Count - P99 Latency");
        foreach (var comparison in Comparisons)
        {
            sb.AppendLine(GenerateRegressionSummary(comparison.Key, comparison.Value));
        }

        return sb.ToString();
    }

    public Dictionary<string, string> GenerateRegressionAnalysisForComparison(string comparisonKey)
    {
        StringBuilder sb = new();
        Dictionary<string, string> csvData = new();
        Dictionary<string, LoadInfo> comparison = Comparisons[comparisonKey];

        string header = "Benchmark,MaxWorkingSetMB,MaxPrivateMemoryMB,RequestsPerMSec,MeanLatencyMS,Latency50thMS,Latency75thMS,Latency90thMS,Latency99thMS";

        // Generate Memory Regressions.
        StringBuilder memRegressions = new();
        memRegressions.AppendLine(header);
        foreach (var benchmark in comparison.Where(c => c.Value.MaxWorkingSetMB >= 10 || c.Value.MaxPrivateMemoryMB >= 10 ))
        {
            memRegressions.AppendLine($"{benchmark.Key},{benchmark.Value.MaxWorkingSetMB},{benchmark.Value.MaxPrivateMemoryMB},{benchmark.Value.RequestsPerMSec},{benchmark.Value.MeanLatencyMS},{benchmark.Value.Latency50thMS},{benchmark.Value.Latency75thMS},{benchmark.Value.Latency90thMS},{benchmark.Value.Latency99thMS}");
        }
        csvData["memory"] = memRegressions.ToString();

        // Generate Throughput Regressions.
        StringBuilder throughputRegressions = new();
        throughputRegressions.AppendLine(header);
        foreach (var benchmark in comparison.Where(c => c.Value.RequestsPerMSec <= -10))
        {
            throughputRegressions.AppendLine($"{benchmark.Key},{benchmark.Value.MaxWorkingSetMB},{benchmark.Value.MaxPrivateMemoryMB},{benchmark.Value.RequestsPerMSec},{benchmark.Value.MeanLatencyMS},{benchmark.Value.Latency50thMS},{benchmark.Value.Latency75thMS},{benchmark.Value.Latency90thMS},{benchmark.Value.Latency99thMS}");
        }
        csvData["throughput"] = throughputRegressions.ToString();

        // Generate Latency Regressions.
        StringBuilder latencyRegressions = new();
        latencyRegressions.AppendLine(header);
        foreach (var benchmark in comparison.Where(c => c.Value.MeanLatencyMS >= 10 || 
                                                        c.Value.Latency50thMS >= 10 || 
                                                        c.Value.Latency75thMS >= 10 || 
                                                        c.Value.Latency90thMS >= 10 || 
                                                        c.Value.Latency99thMS >= 10 ))
        {
            latencyRegressions.AppendLine($"{benchmark.Key},{benchmark.Value.MaxWorkingSetMB},{benchmark.Value.MaxPrivateMemoryMB},{benchmark.Value.RequestsPerMSec},{benchmark.Value.MeanLatencyMS},{benchmark.Value.Latency50thMS},{benchmark.Value.Latency75thMS},{benchmark.Value.Latency90thMS},{benchmark.Value.Latency99thMS}");
        }
        csvData["latency"] = latencyRegressions.ToString();

        // All.
        StringBuilder all = new();
        all.AppendLine(header);
        foreach (var benchmark in comparison)
        {
            all.AppendLine($"{benchmark.Key},{benchmark.Value.MaxWorkingSetMB},{benchmark.Value.MaxPrivateMemoryMB},{benchmark.Value.RequestsPerMSec},{benchmark.Value.MeanLatencyMS},{benchmark.Value.Latency50thMS},{benchmark.Value.Latency75thMS},{benchmark.Value.Latency90thMS},{benchmark.Value.Latency99thMS}");
        }
        csvData["all"] = all.ToString();

        return csvData;
    }

    public void SaveComparisons(string basePath)
    {
        // Add Summary for Comparisons.
        string summaryOfComparisons = GenerateSummaryForComparisons();
        File.WriteAllText(Path.Combine(basePath, "SummaryOfComparisons.csv"), summaryOfComparisons);

        // Add Regression Summary for Comparisons.
        string regressionSummary = GenerateRegressionSummaryForComparisons();
        File.WriteAllText(Path.Combine(basePath, "RegressionSummary.csv"), regressionSummary);

        // Add Large Regression Analysis for Comparison.
        string perComparisonDataPath = Path.Combine(basePath, "PerComparisonData");
        if (!Directory.Exists(perComparisonDataPath))
        {
            Directory.CreateDirectory(perComparisonDataPath);
        }

        foreach (var comparison in Comparisons)
        {
            string comparisonPath = Path.Combine(perComparisonDataPath, comparison.Key);
            Directory.CreateDirectory(comparisonPath);

            Dictionary<string, string> regressionComparisons = GenerateRegressionAnalysisForComparison(comparison.Key);

            // Memory
            File.WriteAllText(Path.Combine(comparisonPath, "MemoryRegressions.csv"), regressionComparisons["memory"]);

            // Throughput
            File.WriteAllText(Path.Combine(comparisonPath, "ThroughputRegressions.csv"), regressionComparisons["throughput"]);

            // Latency
            File.WriteAllText(Path.Combine(comparisonPath, "LatencyRegressions.csv"), regressionComparisons["latency"]);

            // All
            File.WriteAllText(Path.Combine(comparisonPath, "All.csv"), regressionComparisons["all"]);
        }
    }

    public Dictionary<string, Dictionary<string, LoadInfo>> Comparisons { get; }
}

In [ ]:
var datas3_vs_datas_4 = dataManager.GetBenchmarkToComparison("datas_3", "datas_4");

Dictionary<string, Dictionary<string, LoadInfo>> comparisons = new()
{
    { nameof(run1_vs_run2), run1_vs_run2 },
};

SummaryTable summaryTable = new(comparisons);
summaryTable.SaveComparisons("./");

## Charting Helpers

The following cells highlight how to chart certain properties of the LoadInfo class.

In [ ]:
void ChartProperty(LoadInfo baseline, LoadInfo comparand, string nameOfProperty)
{
    GCProcessData baselineGC = baseline.Data;
    GCProcessData comparandGC = comparand.Data;

    List<(string scatterName, List<TraceGC> gcs)> gcData = 
        new()
        {
            { ( scatterName :  $"{nameOfProperty} for {baseline.Id}" , gcs : baselineGC.GCs )},
            { ( scatterName :  $"{nameOfProperty} for {comparand.Id}" , gcs : comparandGC.GCs )}
        };

    GCCharting.ChartGCData(gcData          : gcData, 
                           title           : $"{nameOfProperty} Comparison Between {baseline.Run} and {comparand.Run}", 
                           isXAxisRelative : false,
                           fieldName       : nameOfProperty).Display();

}

void ChartProperty(LoadInfo comparison, string nameOfProperty)
{
    GCProcessData baselineGC = comparison.Data;
    GCProcessData comparandGC = comparison.Data2;

    List<(string scatterName, List<TraceGC> gcs)> gcData = 
        new()
        {
            { ( scatterName :  $"{nameOfProperty} for Baseline" , gcs : baselineGC.GCs )},
            { ( scatterName :  $"{nameOfProperty} for Comparand" , gcs : comparandGC.GCs )}
        };

    GCCharting.ChartGCData(gcData          : gcData, 
                           title           : $"{nameOfProperty} Comparison", 
                           isXAxisRelative : false,
                           fieldName       : nameOfProperty).Display();

}

In [ ]:
dataManager.Data

In [ ]:
var run1_Benchmark = dataManager.GetBenchmarkData(benchmark: "CachingPlatform", "datas_2");
var run2_Benchmark = dataManager.GetBenchmarkData(benchmark: "CachingPlatform", "datas_3");

// Chart the PauseDurationMSec for the run1 vs. run2.
ChartProperty(baseline: run1_Benchmark, comparand: run2_Benchmark, nameof(TraceGC.PauseDurationMSec))

## Debugging

In [ ]:
System.Diagnostics.Process.GetCurrentProcess().Id

In [ ]:
#!about